In [1]:
!pip install pandas openpyxl geopy numpy


In [7]:
# Install required libraries if you haven't already
# !pip install pandas openpyxl geopy numpy

import pandas as pd
import numpy as np
import os
from geopy.distance import geodesic
from geopy.distance import great_circle # Often used for quick straight-line distance

# --- Define Paths and Constants ---

# Root folder path provided by the user
ROOT_FOLDER = r"C:\Users\willi\OneDrive\Documents\COVEX\PYTHON"

# Input data file path
INPUT_FILENAME = os.path.join(ROOT_FOLDER, "zambia_mining_cleaned_FINAL.xlsx")

# Output data file path (for cleaned data to be consumed by the Streamlit app)
OUTPUT_FILENAME = os.path.join(ROOT_FOLDER, "zambia_mining_app_data.csv")

# ----------------------------------------------------------------------
# Define the Base of Operations (Chingola) Coordinates
# Chingola, Zambia: 12°32′S 27°51′E
LAT_CHINGOLA = -12.5333
LON_CHINGOLA = 27.8500
CHINGOLA_COORDS = (LAT_CHINGOLA, LON_CHINGOLA)

print(f"Base of Operations (Chingola) set at: {CHINGOLA_COORDS}")
print("-" * 50)

Base of Operations (Chingola) set at: (-12.5333, 27.85)
--------------------------------------------------


Step 2: Load and Prepare Data
This cell loads your data, renames crucial columns (assuming your Excel sheet has the coordinate columns named correctly or based on the previous conversion), and cleans the district name column.

In [9]:
# Load the data from the specified Excel file
try:
    df = pd.read_excel(INPUT_FILENAME, sheet_name=0)
    print("Data loaded successfully.")
    
    # --- Data Cleaning and Standardization ---
    
    # Standardize ALL required column names for use in Streamlit
    df.rename(columns={
        'Property Name': 'Property_Name',              
        'Locale': 'District/Town',                         
        'Commodity 1': 'Primary_Commodity',                
        'Commodity 2': 'Commodity_2',
        'Commodity 3': 'Commodity_3',
        'Distance From Nearest Center': 'Distance_Nearest_Center_km',
        'Geology Classification': 'Geology_Classification',
        'Geology Description': 'Geology_Description'
        # Latitude and Longitude columns are already clean
    }, inplace=True)
    
    # 1. Clean up Location/District Name Column
    if 'District/Town' in df.columns:
        df['District/Town'] = df['District/Town'].astype(str).str.strip().str.title()
    
    # 2. Clean up Mineral Name Column
    if 'Primary_Commodity' in df.columns:
        df['Primary_Commodity'] = df['Primary_Commodity'].astype(str).str.strip().str.title()

    # 3. Drop rows where coordinates are missing (cannot map them)
    df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

    print(f"Data ready for geospatial preparation. Total valid sites: {len(df)}")
    
    # Display the first few rows to confirm coordinate columns are correct
    print("\nData Head (Check Key Columns):")
    display(df[['Property_Name', 'District/Town', 'Primary_Commodity', 'Latitude', 'Longitude']].head())

except FileNotFoundError:
    print(f"ERROR: File not found at {INPUT_FILENAME}. Please check your path.")
except Exception as e:
    print(f"An error occurred during loading or renaming: {e}")
    print("ACTION REQUIRED: Check the spelling of the column names in the .rename() function against your Excel file's headers.")

Data loaded successfully.
Data ready for geospatial preparation. Total valid sites: 246

Data Head (Check Key Columns):


,Property_Name,District/Town,Primary_Commodity,Latitude,Longitude
0,MNERU WANITPA,A Large Area Between Lakes Mneru Wanitpa And T...,Diamond,-8.252500,29.758333
1,KASANGULA (KASONGOLE),Northwest Of Lake Ameru Vanitipa,Diamond,-8.500000,29.583333
2,CHIMPATIKA (MUERU COPPER),East Of Lake Mueru,Copper,-8.500000,20.166667
3,NCHELENGE,Borders On The Se Shore Of Lake Ameru,Diamond,-9.002500,28.758056
4,CHIBOTE (KANANAMPANGO),51 Km East Of Kananbbua,Iron,-9.833333,29.583333


In [13]:
# --- DEBUGGING CHECK: Find Bad Latitude Values ---
invalid_lat_rows = df[(df['Latitude'] < -90) | (df['Latitude'] > 90)]

if not invalid_lat_rows.empty:
    print("\n\n!! ACTION REQUIRED: INVALID LATITUDE VALUES FOUND !!")
    print("The following rows have Latitude values outside the [-90, 90] range:")
    display(invalid_lat_rows[['Property_Name', 'Latitude', 'Longitude']])
    print("Please correct these values in your Excel file or remove the rows.")
    # Exit or raise error here if you want to stop processing bad data
    
    # We will remove them here to allow the rest of the script to run:
    df.drop(invalid_lat_rows.index, inplace=True)
    print(f"Removed {len(invalid_lat_rows)} rows with invalid Latitude for calculation.")
    print(f"Total valid sites remaining: {len(df)}")
else:
    print("Latitude check passed. All values are within [-90, 90] range.")
# -------------------------------------------------

Latitude check passed. All values are within [-90, 90] range.


Step 3: Calculate Distances and Travel Time
This step calculates the straight-line distance and estimated travel time from the Chingola base to every property.

In [14]:
# Function to calculate Great Circle Distance (straight line)
def calculate_travel_time(row, base_coords, assumed_road_speed_kmph=70):
    # Ensure coordinates are float and valid (redundancy is safety here)
    if pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        return pd.Series([np.nan, np.nan])

    site_coords = (row['Latitude'], row['Longitude'])
    
    # Calculate distance in kilometers
    distance_km = great_circle(base_coords, site_coords).km
    
    # Calculate assumed travel time (Hours)
    travel_time_hours = distance_km / assumed_road_speed_kmph
    
    return pd.Series([distance_km, travel_time_hours])

# --- NEW: Ensure Coordinate Columns are Numeric Floats ---
# This is crucial for geopy, as the error suggests non-numeric values are being treated as invalid coordinates.
print("Cleaning coordinate data types...")
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
print(f"Total valid sites after final coordinate check: {len(df)}")
# ------------------------------------------------------------


# Apply the calculation to the entire DataFrame
df[['Distance_From_Chingola_km', 'Travel_Time_From_Chingola_Hours']] = df.apply(
    calculate_travel_time, 
    axis=1, 
    base_coords=CHINGOLA_COORDS
)

# --- Final Step: Save the Cleaned Data ---

# Save the prepared DataFrame to a CSV file for quick loading by the Streamlit app
df.to_csv(OUTPUT_FILENAME, index=False)

print("-" * 50)
print(f"Final data prepared and saved to CSV: {OUTPUT_FILENAME}")
print("\nFinal Data Head with Travel Planning Columns:")
display(df[['Property_Name', 'District/Town', 'Primary_Commodity', 'Distance_From_Chingola_km', 'Travel_Time_From_Chingola_Hours']].head())

Cleaning coordinate data types...
Total valid sites after final coordinate check: 239
--------------------------------------------------
Final data prepared and saved to CSV: C:\Users\willi\OneDrive\Documents\COVEX\PYTHON\zambia_mining_app_data.csv

Final Data Head with Travel Planning Columns:


,Property_Name,District/Town,Primary_Commodity,Distance_From_Chingola_km,Travel_Time_From_Chingola_Hours
0,MNERU WANITPA,A Large Area Between Lakes Mneru Wanitpa And T...,Diamond,519.730340,7.424719
1,KASANGULA (KASONGOLE),Northwest Of Lake Ameru Vanitipa,Diamond,486.858406,6.955120
2,CHIMPATIKA (MUERU COPPER),East Of Lake Mueru,Copper,952.037774,13.600540
3,NCHELENGE,Borders On The Se Shore Of Lake Ameru,Diamond,404.940247,5.784861
4,CHIBOTE (KANANAMPANGO),51 Km East Of Kananbbua,Iron,354.791484,5.068450
